In [28]:
import librosa
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

def extract_features(file_path):
    # 음성 파일에서 MFCC 및 추가 특징 추출
    y, sr = librosa.load(file_path, sr=None)
    # 음성 파일을 정규화하여 볼륨 차이 줄이기
    y = librosa.util.normalize(y)
    
    # 특징 추출 (MFCC, Chroma, Spectral Contrast, Spectral Bandwidth)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)  # MFCC 개수를 40으로 증가
    delta_mfcc = librosa.feature.delta(mfcc)  # MFCC의 delta 특징 추가
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)  # 추가 특징 추출
    
    # 추출된 특징의 평균값 및 표준편차 계산하여 반환
    features = np.concatenate((
        np.mean(mfcc, axis=1),
        np.std(mfcc, axis=1),
        np.mean(delta_mfcc, axis=1),
        np.std(delta_mfcc, axis=1),
        np.mean(chroma, axis=1),
        np.std(chroma, axis=1),
        np.mean(spectral_contrast, axis=1),
        np.std(spectral_contrast, axis=1),
        np.mean(spectral_bandwidth, axis=1),
        np.std(spectral_bandwidth, axis=1)
    ))
    return features, mfcc

def compare_voices(file1, file2):
    # 두 파일의 특징 추출
    features1, mfcc1 = extract_features(file1)
    features2, mfcc2 = extract_features(file2)
    
    # 동적 시간 왜곡(DTW)을 사용해 MFCC 특징 비교
    distance, _ = fastdtw(mfcc1.T, mfcc2.T, dist=euclidean)
    
    # 코사인 유사도를 사용해 평균 특징 비교
    features1 = features1.reshape(1, -1)
    features2 = features2.reshape(1, -1)
    similarity = cosine_similarity(features1, features2)[0][0]
    
    # 유사도를 0.5 가중치로 합산하여 최종 유사도 계산
    final_similarity = 0.5 * (1 / (1 + distance)) + 0.5 * similarity
    return final_similarity

def is_same_voice(file1, file2, threshold=0.7):
    similarity = compare_voices(file1, file2)
    print(f"Similarity: {similarity:.2f}")
    if similarity > threshold:
        return True
    else:
        return False

# 사용 예시
stored_voice = "C:\Projects\Python_basic\최종프로젝트\빨강.mp3"
new_voice = r"C:\Projects\Python_basic\최종프로젝트\검정.mp3"

if is_same_voice(stored_voice, new_voice):
    print("두 음성은 같은 목소리입니다.")
else:
    print("두 음성은 다른 목소리입니다.")


Similarity: 0.50
두 음성은 다른 목소리입니다.


In [68]:
from resemblyzer import VoiceEncoder, preprocess_wav
from pathlib import Path
import numpy as np

# 모델 로드
encoder = VoiceEncoder()

# 음성 파일 경로 설정
wav_path_1 = Path(r"C:\Projects\Python_basic\최종프로젝트\동현.wav")
wav_path_2 = Path(r"C:\Projects\Python_basic\최종프로젝트\은설.wav")

# WAV 파일 로드 및 전처리
wav_1 = preprocess_wav(wav_path_1)
wav_2 = preprocess_wav(wav_path_2)

# 음성 임베딩 생성
embed_1 = encoder.embed_utterance(wav_1)
embed_2 = encoder.embed_utterance(wav_2)

# 두 벡터 간 코사인 유사도 계산
similarity = np.dot(embed_1, embed_2)
print(f"두 음성의 유사도: {similarity}")

# 유사도가 8.5 이상이면 같은 목소리로 간주
if similarity >= 0.80:
    print("같은 목소리입니다.")
else:
    print("다른 목소리입니다.")


Loaded the voice encoder model on cuda in 0.02 seconds.
두 음성의 유사도: 0.5281274318695068
다른 목소리입니다.


In [2]:
import sounddevice as sd
import wavio

def record_voice(filename="나비.wav", duration=2, samplerate=44100):
    print("Recording...")
    # 음성 녹음 시작
    recording = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=2, dtype='int16')
    sd.wait()  # 녹음 완료될 때까지 대기
    print("Recording finished.")
    
    # 녹음한 음성을 파일로 저장
    wavio.write(filename, recording, samplerate, sampwidth=2)
    print(f"Recording saved as {filename}")

# 예시: 5초 동안 녹음하고 output.wav로 저장
record_voice(duration=3)


Recording...
Recording finished.
Recording saved as 나비.wav
